In [ ]:
!conda install cartopy scipy -c conda-forge -y

In [ ]:
!conda install geemap -c conda-forge -y

In [ ]:
# Cell 1 — environment setup & Earth Engine auth
import os
import ee
import geemap
from geemap import cartoee
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize()
%pylab inline


In [ ]:
# Cell 2 — interactive map base
Map = geemap.Map()
Map


In [ ]:
# Cell 3 — parameters & AOI
lon, lat      = 90.478, 22.942
start_year    = 2017
end_year      = 2024
point         = ee.Geometry.Point(lon, lat)
years         = list(range(start_year, end_year + 1))  # Python list


In [ ]:
# Cell 4 — define region, viz bands & styling dicts
# half-width & half-height
w, h    = 0.6, 0.56

# note: for your workflow you used [east, south, west, north]
# but EE expects [west, south, east, north] here, so we flip:
region  = [lon - w, lat - h, lon + w, lat + h]  

# which embedding axes will map to R, G, B
viz_bands = ['A01', 'A16', 'A09']

# grid / north‐arrow / scale‐bar parameters
north_arrow_dict = {
    "text": "N", "xy": (0.1, 0.3), "arrow_length": 0.15,
    "text_color": "white", "arrow_color": "white",
    "fontsize": 20, "width": 5, "headwidth": 15,
    "ha": "center", "va": "center"
}
scale_bar_dict = {
    "length": 10, "xy": (0.1, 0.05), "linewidth": 3,
    "fontsize": 20, "color": "white", "unit": "km",
    "ha": "center", "va": "bottom"
}


In [ ]:
# Cell 5 — function to pull & 2%–98% stretch each year + build collection
def get_stretched_embedding(year):
    # grab the first embedding image of that year
    img = (ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
             .filterBounds(point)
             .filterDate(ee.Date.fromYMD(year, 1, 1),
                         ee.Date.fromYMD(year,12,31))
             .first())
    # compute the 2nd & 98th percentile of each viz band over the region
    stats = img.select(viz_bands).reduceRegion(
        reducer    = ee.Reducer.percentile([2, 98]),
        geometry   = ee.Geometry.Rectangle(region),
        scale      = 30,
        maxPixels  = 1e13,
        bestEffort = True
    )
    mins = [stats.get(f"{b}_p2")  for b in viz_bands]
    maxs = [stats.get(f"{b}_p98") for b in viz_bands]
    # bake in the stretch and tag with year
    return img.visualize(bands=viz_bands, min=mins, max=maxs) \
              .set('year', int(year))

# build Python list + wrap into an EE collection
images     = [get_stretched_embedding(y) for y in years]
collection = ee.ImageCollection(images)

# debug
print("Years in collection:", collection.aggregate_array('year').getInfo())


In [ ]:
# Cell 6 — add the first frame to your interactive map
first = ee.Image(collection.filter(ee.Filter.eq('year', years[0])).first())
Map.setCenter(lon, lat, 10)
Map.addLayer(first, {}, f'Embedding {years[0]} (2 % stretch)')
Map


In [ ]:
# Cell 7 — static plot with grid, north‐arrow & scale‐bar
stretched = ee.Image(collection.filter(ee.Filter.eq('year', years[0])).first())

fig = plt.figure(figsize=(10, 10))
ax  = cartoee.get_map(stretched, region=region)  # no vis_params needed

cartoee.add_gridlines(ax, interval=[0.2, 0.2], linestyle=":")
cartoee.add_north_arrow(ax, **north_arrow_dict)
cartoee.add_scale_bar_lite(ax, **scale_bar_dict)

ax.set_title(f"Google Embedding axes {viz_bands} — {years[0]} (2 % stretch)", fontsize=15)
plt.show()


In [ ]:
# Cell 8 — Generate annotated PNG frames, shifted right by 0.1° longitude

import os
import matplotlib.pyplot as plt
from geemap import cartoee
import ee

# Re-define key variables
years = list(range(2017, 2025))
lon, lat = 90.478, 22.942
w, h = 0.6, 0.56
# original bbox [west, south, east, north]
region = [lon - w, lat - h, lon + w, lat + h]

north_arrow_dict = {
    "text": "N", "xy": (0.1, 0.3), "arrow_length": 0.15,
    "text_color": "white", "arrow_color": "white",
    "fontsize": 20, "width": 5, "headwidth": 15,
    "ha": "center", "va": "center"
}
scale_bar_dict = {
    "length": 10, "xy": (0.1, 0.05), "linewidth": 3,
    "fontsize": 20, "color": "white", "unit": "km",
    "ha": "center", "va": "bottom"
}

output_dir = os.path.expanduser("./Downloads")
os.makedirs(output_dir, exist_ok=True)
print("Saving annotated frames to:", output_dir)

dx = 0.2  # degrees to shift right

for year in years:
    img = ee.Image(collection.filter(ee.Filter.eq('year', year)).first())
    
    # Shift the region right
    region_shift = [region[0] + dx, region[1], region[2] + dx, region[3]]
    
    # Create a matplotlib figure
    fig = plt.figure(figsize=(10, 8))
    ax = cartoee.get_map(img, region=region_shift)
    
    # Add overlays
    cartoee.add_gridlines(ax, interval=[0.2, 0.2], linestyle=":")
    cartoee.add_north_arrow(ax, **north_arrow_dict)
    cartoee.add_scale_bar_lite(ax, **scale_bar_dict)
    
    # Labels, title, and fix orientation
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title(f"Embedding {year}", fontsize=16)
    #ax.invert_yaxis()
    
    # Save the annotated frame
    out_path = os.path.join(output_dir, f"frame_{year}.png")
    fig.savefig(out_path, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("Saved:", out_path)


In [ ]:
9 # Create GIF
gif_path = os.path.join(annotated_dir, "embedding_annotated_animation.gif")
png_to_gif(annotated_dir, gif_path, fps=2)
print("GIF saved to:", gif_path)
display(Image(filename=gif_path))

# Now also make an MP4
!ffmpeg -y -i {gif_path} \
  -movflags faststart \
  -pix_fmt yuv420p \
  {gif_path.replace('.gif','.mp4')}

# And preview it
from IPython.display import Video
Video(gif_path.replace('.gif','.mp4'), embed=True)
